## Imports

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt

plt.style.use('seaborn')
%matplotlib inline

## Data Preprocessing

In [ ]:
%%time
df = pd.read_csv('data/yesbank_data.csv', index_col=0, parse_dates=True)

In [ ]:
df.rename(columns={'close': 'c', 'open':'o', 'high':'h', 'low':'l', 'volume':'v'}, inplace=True)

In [ ]:
df.head()

In [ ]:
df['c'].plot()

## Implementing a simple crossover strategy

In [ ]:
df['r'] = np.log(df['c'] / df['c'].shift(1))
df['sma1'] = df['c'].rolling(15).mean()
df['sma2'] = df['c'].rolling(30).mean()
df['sma3'] = df['c'].rolling(60).mean()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['market_dir'] = np.where(df['r'] > 0, 1, -1)

In [ ]:
df['strat_dir'] = np.where(df['sma1'] > df['sma3'], 1, -1)

In [ ]:
df.head(10)

In [ ]:
df['strat_dir'].diff().value_counts()

In [ ]:
df['s'] = df['strat_dir'] * df['r']

In [ ]:
df[['r', 's']].cumsum().apply(np.exp).plot(figsize=(10, 10));

## Add a machine learning model and perform accuracy testing

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
mdf = df.copy()

In [ ]:
for i in range(1,6):
     mdf[f'lag_{i}'] = mdf['market_dir'].shift(i)

In [ ]:
mdf.dropna(inplace=True)
mdf.head()

In [ ]:
X = mdf[['lag_1','lag_2', 'lag_3', 'lag_4','lag_5']]

In [ ]:
y = mdf['market_dir']

In [ ]:
X_train = X[:500]
X_test = X[500:]
y_train = y[:500]
y_test = y[500:]

In [ ]:
classifier = LogisticRegression()

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear')

In [ ]:
classifier.fit(X_train,y_train)

## In Sample testing

In [ ]:
y_pred = classifier.predict(X_train)
accuracy_score(y_pred, y_train)

## Holdout Testing

In [ ]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test,y_pred)